In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load the data
data = pd.read_csv('output.csv')

# Drop non-numeric columns
data = data.drop(columns=['Address','Status', 'Data Date'])


In [5]:
X = data.values[:, :-1]  # All but the last month
y = data.values[:, -1]  # Last month

X = X.reshape((X.shape[0], X.shape[1], 1))
max_months = data.shape[1] - 1  # Subtract 1 to exclude the target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(max_months//2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(max_months//4, activation='relu'),
    tf.keras.layers.Dense(1)  # Linear activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
2155/2155 [==============================] - 18s 8ms/step - loss: 21847787520.0000 - mae: 31409.0156 - val_loss: 2255645184.0000 - val_mae: 13263.4648
Epoch 2/20
2155/2155 [==============================] - 17s 8ms/step - loss: 7754586112.0000 - mae: 15348.5869 - val_loss: 2147983104.0000 - val_mae: 11832.0361
Epoch 3/20
2155/2155 [==============================] - 17s 8ms/step - loss: 2771962368.0000 - mae: 12760.1504 - val_loss: 1940516224.0000 - val_mae: 12431.3418
Epoch 4/20
2155/2155 [==============================] - 18s 9ms/step - loss: 1733204480.0000 - mae: 13292.0293 - val_loss: 1521170560.0000 - val_mae: 10524.4238
Epoch 5/20
2155/2155 [==============================] - 18s 9ms/step - loss: 1188672896.0000 - mae: 12571.5957 - val_loss: 2096932992.0000 - val_mae: 18790.1465
Epoch 6/20
2155/2155 [==============================] - 18s 9ms/step - loss: 1358007936.0000 - mae: 11942.2666 - val_loss: 1549769856.0000 - val_mae: 12143.4355
Epoch 7/20
2155/2155 [===========

In [6]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')

# Make predictions on the validation data
y_pred = model.predict(X_test)

# If you want to compute R-squared (R²) on validation data
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f'R-squared (R²) on validation data: {r2}')

# If you want to predict on the entire dataset
predictions_full = model.predict(X.reshape((X.shape[0], X.shape[1], 1)))  # Ensure X is 3D
predictions_full_df = pd.DataFrame(predictions_full, columns=['Predicted Price'])
print(predictions_full_df)

1078/1078 [==============================] - 2s 2ms/step - loss: 1431531264.0000 - mae: 9176.9678
Mean Absolute Error: 9176.9677734375
1078/1078 [==============================] - 2s 2ms/step
R-squared (R²) on validation data: 0.9962669480241106
5386/5386 [==============================] - 10s 2ms/step
        Predicted Price
0         122774.273438
1         238399.703125
2         294029.531250
3         304463.750000
4         276552.187500
...                 ...
172336    577045.062500
172337    908520.937500
172338    399831.468750
172339    464117.218750
172340    993272.562500

[172341 rows x 1 columns]


In [4]:
model.save("saved_model/my_model")

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets
